In [2]:
import os

In [4]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz

In [8]:
paths = {
    'WORKSPACE_PATH': os.path.join('data', 'workspace'),
    'SCRIPTS_PATH': os.path.join('data','scripts'),
    'APIMODEL_PATH': os.path.join('data','models'),
    'ANNOTATION_PATH': os.path.join('data', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('data', 'workspace','images'),
    'MODEL_PATH': os.path.join('data', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('data', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('data', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('data', 'workspace','models',CUSTOM_MODEL_NAME, 'export'),  
    'PROTOC_PATH':os.path.join('data','protoc')
 }

NameError: name 'CUSTOM_MODEL_NAME' is not defined

In [3]:
files = {
    'PIPELINE_CONFIG':os.path.join('data', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

NameError: name 'CUSTOM_MODEL_NAME' is not defined

In [6]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

/bin/bash: /home/vojta/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2023-02-28 21:45:19.616456: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-28 21:45:19.799394: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
^C
Traceback (most recent call last):
  File "/mnt/c/VUTFIT/BP/code/api/data/models/research/object_detection/builders/model_builder_tf2_test.py", line 21, in <module>
    import tensorflow.compat.v1 as tf
  File "/home/vojta/

In [6]:

import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [7]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [8]:
config

{'model': ssd {
   num_classes: 1
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   match

In [9]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()

with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)  

In [10]:
pipeline_config.model.ssd.num_classes = 1
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.tfrec')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'val.tfrec')]

In [11]:
config_text = text_format.MessageToString(pipeline_config)      

with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:
    f.write(config_text)

In [12]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [13]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])
print(command)

python data/models/research/object_detection/model_main_tf2.py --model_dir=data/workspace/models/my_ssd_mobnet --pipeline_config_path=data/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=2000


In [59]:
!{command}

/bin/bash: /home/vojta/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2023-02-25 20:40:59.669961: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-25 20:40:59.984543: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-25 20:41:01.438017: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file 

In [39]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util
# Load pipeline config and build a detection model
#configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
configs = config_util.get_configs_from_pipeline_file(os.path.join('data', 'workspace','models','ssd640', "pipeline.config"))
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
#ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-24'))
ckpt.restore(os.path.join('data', 'workspace','models','ssd640', 'ckpt-235'))


@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [40]:
import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
category_index = label_map_util.create_category_index_from_labelmap(os.path.join('data', 'workspace','annotations', 'label_map.pbtxt'))
#IMAGE_PATH = "/mnt/c/VUTFIT/BP/code/api/data/workspace/aug/val/images/5924.png"
IMAGE_PATH = "/mnt/c/VUTFIT/BP/code/data/test2.png"
#IMAGE_PATH = "/mnt/c/VUTFIT/BP/code/api/data/workspace/aug/train/images/12322.jpg"
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)


num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=100,
            min_score_thresh=.3,
            agnostic_mode=False)

plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.show()


2023-03-24 16:30:22.372020: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at save_restore_v2_ops.cc:227 : DATA_LOSS: TensorBundle at data/workspace/models/ssd640/ckpt-235 shard 0 (98304 bytes): Checksum does not match: stored 3424403450 vs. calculated on the restored bytes 329430801


DataLossError: in user code:

    File "/tmp/ipykernel_1932/2280237598.py", line 21, in detect_fn  *
        prediction_dict = detection_model.predict(image, shapes)
    File "/home/vojta/miniconda3/envs/tf/lib/python3.10/site-packages/object_detection/meta_architectures/ssd_meta_arch.py", line 570, in predict  *
        feature_maps = self._feature_extractor(preprocessed_inputs)
    File "/home/vojta/miniconda3/envs/tf/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/home/vojta/miniconda3/envs/tf/lib/python3.10/site-packages/object_detection/models/ssd_mobilenet_v2_fpn_keras_feature_extractor.py", line 150, in build
        self.classification_backbone = tf.keras.Model(

    DataLossError: TensorBundle at data/workspace/models/ssd640/ckpt-235 shard 0 (98304 bytes): Checksum does not match: stored 3424403450 vs. calculated on the restored bytes 329430801 [Op:RestoreV2]
